# Analise de ações com algoritmos de forecast

### instalando as dependecias

In [ ]:
!pip install yfinance
!pip install fbprophet

## Importa dependecias

In [3]:
import pandas as pd
import yfinance as yf
import datetime 
import numpy as np
from datetime import timedelta
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings
warnings.filterwarnings('ignore')



ModuleNotFoundError: No module named 'fbprophet'

## Seleciona o código da ação e periodo

In [ ]:
tickers = 'ARZZ3.SA'
start_date = datetime.datetime.now().date() + datetime.timedelta(days=-4000)
end_date = datetime.datetime.now().date() 

## Faz o download do dataframe com as informações

In [ ]:
df_ticket = yf.download(tickers,start_date,end_date)

In [ ]:
df_ticket.reset_index(inplace=True)

## Analise do dataframe

In [ ]:
#print(df_ticket.isnull().sum())
#print(df_ticket.loc[df_ticket['Adj Close']==0])
print(df_ticket.describe)


## Ajuste tipo da Data

In [ ]:
df_ticket.Date = pd.to_datetime(df_ticket.Date)

In [ ]:
df_ticket = df_ticket.set_index(df_ticket['Date']).sort_index()
df_ticket.drop("Date", axis=1, inplace=True)

## Visualiza gráfico

In [ ]:
df_ticket.info
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_ticket.index, y=df_ticket['Close']))


In [ ]:
ts = df_ticket[['Open','Adj Close']]
ts.plot(figsize=(20,8))

## Media Móvel

In [ ]:
df_ticket['MediaMovel10Dias'] = df_ticket["Adj Close"].rolling(window=10).mean()
df_ticket['MediaMovel30Dias'] = df_ticket["Adj Close"].rolling(window=30).mean()
df_ticket['MediaMovel150Dias'] = df_ticket["Adj Close"].rolling(window=150).mean()
df_ticket['MediaMovel365Dias'] = df_ticket["Adj Close"].rolling(window=365).mean()

In [ ]:
ts = df_ticket[['Open','Adj Close','MediaMovel10Dias','MediaMovel30Dias','MediaMovel150Dias','MediaMovel365Dias']]

ts.plot(figsize=(20,8))

# Forecast

## Prophet 

https://facebook.github.io/prophet/docs/quick_start.html

### configura a base de treino do modelo

In [ ]:
#Vamos separar a base no periodo de teste e treino

# Inicalmente vamos treinar o modelo de 05/1997 a 05/2019
start_train = "2011-05-14"
end_train = "2021-11-24"

#Vamos testar o modelo de 05/2019 a 06/2020
start_test = "2019-06-01"
end_test = "2020-06-05"

df_train = df_ticket[start_train : end_train]
df_test = df_ticket[start_test : end_test]

print('Train Dataset:',df_train.shape)
print('Test Dataset:',df_test.shape)
print(df_train.head())


In [ ]:
df_test = df_ticket

#Transformando o Dataframe de teste correto para o Prophet
#Precisamos que a data volte a ser uma coluna
df_test = df_test.reset_index() 

#Excluindo as colunas que não vamos precisar
df_test.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1, inplace = True) 

#Renomeando as colunas para o padrão do prophet
df_test.rename(columns={'Close': 'y', 'Date': 'ds'}, inplace=True) 
df_test.head()

### executo script de treino do modelo

In [ ]:
#Transformando o dataset de treinamento correto para o Prophet

df_train = df_train.reset_index()

df_train.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1, inplace = True)

df_train.rename(columns={'Close': 'y', 'Date': 'ds'}, inplace=True)

df_train.head()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_test['ds'], y=df_test['y']))


### Aplica o modelo

In [ ]:
model = Prophet(growth ="linear",
                seasonality_mode = 'multiplicative',
                daily_seasonality = False)
model.fit(df_train)

In [ ]:
# determinar o período de dias para previsão
future_prices = model.make_future_dataframe(periods=365)

#Excluindo os finais de semana da  predição
future = future_prices[future_prices['ds'].dt.dayofweek < 5]

# realizar a previsão para o período estabelecido
forecast = model.predict(future)

No Dataframe resultante da predição do Modelo do Prohet há 4 colunas, além da Data, são elas:

yhat = valor predito

yhat_lower = Menor valor predito

yhat_upper = Maior valor predito

e sem mais delongas, vamos visualizar o gráfico da previsão.

### Analise do resultado

In [ ]:
# É possivel plotar um grafico direto da biblioteca prophet
model.plot(forecast, xlabel='Data', ylabel='Preço');

In [ ]:
plot_plotly(model,forecast)

## Average forecasting

O average forecasting assume que os dados estão distribuídos em torno de uma média constante.
Rigorosamente, esse tipo de série temporal é chamada estacionária.

https://www.letscode.com.br/blog/python-para-financas-analise-de-series-temporais

### 

In [ ]:
def treino_teste(dados):

    # Definir quantos porcento queremos pra treino
    pct_treino = 0.75


    # Definir o primeiro dia, o último e o total de dias nos nossos dados
    primeiro_dia = min(dados.index)
    ultimo_dia = max(dados.index)
    total_dias = (ultimo_dia - primeiro_dia).days

    
    # "Dias de treino, dias de glória" - Charlie Brown Jr
    dias_treino = np.ceil(total_dias * pct_treino)
    dias_teste = np.floor(total_dias * (1 - pct_treino))

    # Data em que o treino deve parar
    data_parada = primeiro_dia + timedelta(days=dias_treino)
    

    # Dividir o dataset usando a data de parada do treino
    dados_treino = dados[:data_parada] # do começo à data de parada
    dados_teste = dados[data_parada:] # da data de parada ao final

    return dados_treino, dados_teste, data_parada

In [ ]:
def AF(X, y):
    # Criando lista com as médias
    y_hat_AF = []

    for i in range(len(y)):
        y_hat_AF.append(np.mean(X))

    # Transformando em Pandas Series
    y_hat_AF = pd.Series(y_hat_AF, index=y.index)
    
    return y_hat_AF

In [ ]:
X_ticket, y_ticket, data_parada_ticket = treino_teste(df_ticket['Close'])    

In [ ]:
y_hat_AF_ticket = AF(X_ticket, y_ticket)
df_average_forecasting = y_hat_AF_ticket

In [ ]:
# Gerando gráfico com fundo escuro

with plt.style.context('dark_background'):
    plt.figure(figsize=(20, 5.5))
    plt.title("Average Forecast")
  
    # Dados reais
    plt.plot(X_ticket, label='Valores de treino')
    plt.plot(y_ticket,label='Valores reais')

    

    # Predições
    plt.plot(y_hat_AF_ticket, label='Average forecasting', color='red')
    plt.legend()
    plt.show()

## Drift forecast

O drift forecast assume que os dados estão distribuídos em torno de uma média constante que aumenta ou diminui constantemente no tempo.

Rigorosamente, esse tipo de série temporal é dito com drift.

In [ ]:
def DF(X, y):
    y_t = X[-1]
    m = (y_t - X[0]) / len(X)
    h = np.linspace(0,len(y.index)-1, len(y.index))

    # Criando lista com o resultado da fórmula
    y_hat_DF = []
    
    for i in range(len(y.index)):
        y_hat_DF.append(y_t + m * h[i])

    # Transformando em Pandas Series
    y_hat_DF = pd.Series(y_hat_DF, index=y.index)

    return y_hat_DF

In [ ]:
X_drift_ticket, y_drift_ticket, data_parada_drift_ticket = treino_teste(df_ticket['Close'])

In [ ]:
# Prevendo dados futuros com a função
y_hat_AF__drift_ticket = AF(X_drift_ticket, y_drift_ticket)

In [ ]:
# Prevendo dados futuros com a função
y_hat_DF_drift_ticket = DF(X_drift_ticket, y_drift_ticket)

In [ ]:
# Gerando gráfico com fundo escuro

with plt.style.context('dark_background'):
    plt.figure(figsize=(20, 5.5))
    plt.title("Average Forecast")

    # Dados reais
    plt.plot(X_drift_ticket, label='Valores de treino')
    plt.plot(y_drift_ticket, label='Valores reais')

    # Predições
    plt.plot(y_hat_DF_drift_ticket, label='Average Forecast', color='red')
    plt.plot(y_hat_DF_drift_ticket, label='Drift Forecast', color='Yellow')

    plt.legend()
    plt.show()

## Simple moving average

O simple moving average acompanha as médias se movendo ao longo do tempo.

In [ ]:
def SMA(dados, day):
    y_hat_SMA = dados['Close'].rolling(window=day).mean()

    return y_hat_SMA

In [ ]:
days = [5, 10, 25, 50]
colors = ['green', 'blue', 'pink', 'purple']

# Gerando gráfico com fundo escuro 
with plt.style.context('dark_background'):
    # Configurações básicas
    plt.figure(figsize=(20, 5.5))
    plt.title("Simple Moving Average")

    # Dados reais
    plt.plot(X_ticket, label='Train values', color='white')
    plt.plot(y_ticket, label='Real value')

    # Moving average
    for i, day in enumerate(days):
        y_hat_SMA_ticket = SMA(df_ticket, day)
        plt.plot(y_hat_SMA_ticket, label='Simple moving average '+str(day), color=colors[i])

    # Previsões anteriores        
    plt.plot(y_hat_AF_ticket, label='Average Forecast', color='red')
    plt.plot(y_hat_DF_drift_ticket, label='Drift Forecast', color='Yellow')


    # Configurações básicas
    plt.legend()
    plt.show()

## Cumulative moving average